In [1]:
import os
import torch
import gdown
import numpy as np
import scanpy as sc
import pandas as pd
import scarches as sca
import matplotlib.pyplot as plt

 captum (see https://github.com/pytorch/captum).
INFO:lightning_fabric.utilities.seed:Global seed set to 0
/home/alextopalova/.local/lib/python3.9/site-packages/flax/struct.py:132: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)
/home/alextopalova/.local/lib/python3.9/site-packages/flax/struct.py:132: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)


In [2]:
early_stopping_kwargs = {
    'early_stopping_metric': 'val_prototype_loss',
    'mode': 'min',
    'threshold': 0,
    'patience': 20,
    'reduce_lr': True,
    'lr_patience': 13,
    'lr_factor': 0.1,
}

condition_key = 'study'
cell_type_key = ['cell_type']

In [3]:
adata = sc.read('mouse_pancreas_subsample.h5ad')

In [4]:
adata

AnnData object with n_obs × n_vars = 17353 × 31706
    obs: 'study_sample', 'study', 'file', 'reference', 'size_factors_sample', 'phase_cyclone', 's_cyclone', 'g2m_cyclone', 'g1_cyclone', 'sex', 'ins_score', 'ins_high', 'gcg_score', 'gcg_high', 'sst_score', 'sst_high', 'ppy_score', 'ppy_high', 'cell_filtering', 'age', 'strain', 'tissue', 'technique', 'study_sample_design', 'cell_type', 'cell_type_multiplet', 'cell_subtype', 'cell_subtype_multiplet', 'design', 'size_factors_integrated', 'pre_cell_type_unified', 'pre_cell_type_original', 'study_parsed', 'cell_type_parsed', 'low_q', 'BETA-DATA_leiden_r1.5', 'BETA-DATA_leiden_r20', 'BETA-DATA_hc_gene_programs', 'BETA-DATA_hc_gene_programs_parsed', 'BETA-DATA_leiden_r1.5_parsed', 'BETA-DATA_leiden_r1.5_parsed_const', 'CXG-DATA_n_genes', 'CXG-DATA_mt_frac', 'CXG-DATA_doublet_score', 'CXG-DATA_log10_n_counts', 'CXG-DATA_age_approxDays', 'CXG-DATA_cell_subtype_immune_reannotatedIntegrated', 'CXG-DATA_cell_subtype_endothelial_reannotatedIntegra

In [5]:
np.unique(adata.obs[condition_key])

array(['Fltp_adult'], dtype=object)

In [6]:
# Without this line a TypeError is thrown
adata.obs['cell_type'] = adata.obs['cell_type'].astype(str)

In [7]:
scpoli_model = sca.models.scPoli(
    adata=adata.copy(),
    condition_key='study',
    cell_type_keys=['cell_type'],
    embedding_dim=3,
)

Embedding dictionary:
 	Num conditions: 1
 	Embedding dim: 3
Encoder Architecture:
	Input Layer in, out and cond: 31706 256 3
	Hidden Layer 1 in/out: 256 64
	Mean/Var Layer in/out: 64 10
Decoder Architecture:
	First Layer in, out and cond:  10 64 3
	Hidden Layer 1 in/out: 64 256
	Output Layer in/out:  256 31706 



In [8]:
scpoli_model.train( 
    n_epochs=50,
    pretraining_epochs=40,
    early_stopping_kwargs=early_stopping_kwargs,
    eta=5,
)

RuntimeError: expected scalar type Float but found Double

In [ ]:
scpoli_model.save('/scpoli_pancreas', overwrite =True)